In [2]:
# 데이터셋 출처: https://aihub.or.kr/aidata/7978
# 참고 코드: https://wikidocs.net/50739

In [32]:
import pandas as pd
import matplotlib.pyplot as plt
from gensim.models.word2vec import Word2Vec
from konlpy.tag import Hannanum, Kkma, Komoran, Twitter, Okt
hnn, kkma, komo, tw, okt = Hannanum(), Kkma(), Komoran(), Twitter(), Okt()
import re

In [66]:
train_df = pd.read_excel('/ntdev/youngri/YR_LAB/embedding_search/W2V/corpus/감성대화말뭉치(최종데이터)_Training.xlsx')
train_df.drop(columns=['번호','연령', '성별', '상황키워드', '신체질환','감정_대분류','감정_소분류'], inplace=True)
valid_df = pd.read_excel('/ntdev/youngri/YR_LAB/embedding_search/W2V/corpus/감성대화말뭉치(최종데이터)_Validation.xlsx')
valid_df.drop(columns=['번호','연령', '성별', '상황키워드', '신체질환','감정_대분류','감정_소분류'], inplace=True)
print(train_df.shape, valid_df.shape)
total_df = pd.concat([train_df,valid_df])
print(total_df.shape)

total_df.head()

(40879, 8) (5130, 8)
(46009, 8)


,사람문장1,시스템응답1,사람문장2,시스템응답2,사람문장3,시스템응답3,사람문장4,시스템응답4
0,아내가 드디어 출산하게 되어서 정말 신이 나.,아내분이 출산을 하시는군요. 정말 축하드려요.,아 지금 정말 신이 나.,잘 된 일이네요.,아기가 점점 클게 벌써 기대가 되네. 내가 많이 놀아줘야지.,좋은 아빠가 되실 거 같아요. 진심으로 축하드려요.,NaN,NaN
1,당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야.,약 종류가 많아 번거로우시겠어요.,건강할 때 관리 좀 잘할걸 하는 생각이 들더라고.,현재 상황에서 변화를 주기 위해 어떻게 하면 될까요?,약을 잘 챙겨 먹고 나을 수 있도록 노력해야지.,약을 잘 챙겨 먹어 건강해지시길 바랄게요.,NaN,NaN
2,고등학교에 올라오니 중학교 때보다 수업이 갑자기 어려워져서 당황스러워.,고등학교 수업이 중학교 때와는 다르게 갑자기 어려워서 당황스러우시군요.,아직 학기 초인데 내가 수업에 잘 따라갈 수 있을지 걱정돼.,이 상황에서 할 수 있는 게 어떤 것이 있을까요?,중학교 수업을 들을 때보다 훨씬 더 열심히 할 거야.,수업을 어떤 방식으로 열심히 듣는게 좋을까요?,선생님이 강조하신 부분을 필기하고 집에서 매일 수업 내용을 복습하려고 해.,열심히 해서 좋은 결과가 있었으면 좋겠어요.
3,재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야. 너무 행복해.,재취업 후 첫 월급이라 정말 기쁘시겠어요.,퇴직 후 다시는 돈을 못 벌 줄 알았는데 이렇게 월급으로 가족에게 맛있는 밥을 살 ...,이 기분을 유지하려면 어떻게 해야 할까요?,회사생활을 열심히 해서 계속 월급을 받을거야!,계속 원하시는 대로 회사생활이 이어지길 바랄게요.,NaN,NaN
4,빚을 드디어 다 갚게 되어서 이제야 안도감이 들어.,기분 좋으시겠어요. 앞으로는 어떻게 하실 생각이신가요?,빚도 다 갚았으니 당분간은 아무 생각도 안 하며 살고 싶어.,빚도 다 갚았으니 당분간은 아무 생각도 안 하며 살고 싶으시군요.,NaN,NaN,NaN,NaN


In [57]:
# 테스트로 진행

# len_r, len_c = 2, 8
# test_sents=[]

# for r in range(len_r):
#     for c in range(len_c):
#         if type(train_df.iloc[r][c]) == str:
#             new_sent = re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","",train_df.iloc[r][c])
#             test_sents.append(kkma.morphs(new_sent))

# test_sents

# orga = [hnn, kkma, komo, tw, okt]
# [i.morphs(test_sents[0]) for i in orga]
# 일단 kkma 으로 진행

# model = Word2Vec(sentences = test_sents, vector_size = 100, window = 5, min_count = 1, workers = 5, sg = 0)
# model.wv.key_to_index

In [67]:
len_r, len_c = total_df.shape
# (46009, 8)

use_sents=[]

for r in range(len_r):
    if r % 100 == 0 : print(r*100/len_r)
    for c in range(len_c):
        if type(train_df.iloc[r][c]) == str:
            new_sent = re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","",train_df.iloc[r][c])
            use_sents.append(kkma.morphs(new_sent))

print('--done--')

0.0
0.21734877958660262
0.43469755917320524
0.6520463387598079
0.8693951183464105
1.086743897933013
1.3040926775196158
1.5214414571062183
1.738790236692821
1.9561390162794237
2.173487795866026
2.390836575452629
2.6081853550392315
2.8255341346258342
3.0428829142124365
3.260231693799039
3.477580473385642
3.6949292529722446
3.9122780325588473
4.12962681214545
4.346975591732052
4.564324371318655
4.781673150905258
4.99902193049186
5.216370710078463
5.433719489665066
5.6510682692516685
5.868417048838271
6.085765828424873
6.303114608011476
6.520463387598078
6.737812167184681
6.955160946771284
7.1725097263578865


KeyboardInterrupt: 

In [69]:
model = Word2Vec(sentences = use_sents, vector_size = 100, window = 5, min_count = 5, workers = 4, sg = 0)
model.wv.vectors.shape

(2575, 100)

In [92]:
print(model.wv.most_similar('걱정', topn=10))

[('후회', 0.8057870864868164), ('느낌', 0.7538537383079529), ('고민', 0.7506385445594788), ('다행', 0.7215479016304016), ('부담', 0.6975568532943726), ('낙담', 0.6870628595352173), ('고립', 0.6400182843208313), ('지경', 0.634565532207489), ('안심', 0.6284563541412354), ('불안', 0.627149760723114)]


In [82]:
sample = '너무 슬퍼서 눈물이 나'
sample_morphs = kkma.morphs(sample)

for s in sample_morphs:
    print('-'*50)
    print(s)
    print([i[0] for i in model.wv.most_similar(s, topn=5)])

--------------------------------------------------
너무
['정말', '네', '눈물', '그래서', '원망']
--------------------------------------------------
슬프
['속상하', '억울', '기쁘', '울', '괴롭']
--------------------------------------------------
어서
['어', 'ㄴ다니', '네', '구나', 'ㄴ다']
--------------------------------------------------
눈물
['짜증', '서럽', '심해', '고생', '화도']
--------------------------------------------------
이
['소외', '고립', '사그라지', '수도', '아프']
--------------------------------------------------
나
['자신', '다', '날', '못', '이제']
